# 使用API快速搭建你的第一个AI应用
> [HW3：以 AI 搭建自己的應用](https://colab.research.google.com/drive/15jh4v_TBPsTyIBhi0Fz46gEkjvhzGaBR?usp=sharing)中文镜像版

目标：

了解如何通过使用 API 和 Prompt 来构建自己的语言模型应用。

# 安装库

In [ ]:
!pip3 install openai
!pip3 install gradio

# 导入与设置

In [ ]:
import os
import json
from typing import List, Dict, Tuple

import openai
import gradio as gr

In [ ]:
# TODO: 设置你的 OPENAI API 密钥，这里以阿里云 DashScope API为例进行演示
OPENAI_API_KEY = ""
client = openai.OpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",  # 使用阿里云大模型API
)

# 检查是否正确设置了 API
# 如果一切正常，你将看到 "API 设置成功！！"
try:
    response = client.chat.completions.create(
            model="qwen-turbo",  # 使用阿里云 DashScope 的模型
            messages=[{'role': 'user', 'content': "测试"}],  # 设置一个简单的测试消息
            max_tokens=1,
    )
    print("API 设置成功！！")  # 输出成功信息
except Exception as e:
    print(f"API 可能有问题，请检查：{e}")  # 输出详细的错误信息

## 第1部分：文章摘要（单轮对话应用）

在此任务中，你需要将你的聊天机器人变为一个**摘要器**。它的工作是当用户输入一篇文章时，能够为用户总结该文章的内容。

你需要完成以下步骤：

1. 设计一个用于生成摘要的提示词，并填写在 **prompt_for_summarization** 中。
2. **点击运行按钮**， 这将弹出一个可交互的界面。
3. 你可以找到一篇文章或使用当前的示例文章：《从百草园到三味书屋》，并将其填写在标记为“文章”的输入框中。
4. 点击“发送”按钮生成文章的摘要。（你可以使用“温度”滑块来控制输出的创造性，温度越高，输出越具创造性）。
5. 如果你**想更改提示词**，可以停止单元格，返回到TODO部分进行更改，然后再次运行。
7. 在你获得满意的结果后，点击“导出”按钮保存结果。文件列表中将出现一个名为 **part1.json** 的文件。

注意：

- **如果你再次点击“导出”按钮，之前的结果将被覆盖。**
- **即使使用相同的提示词，输出的结果可能仍然不同。**

------

在运行此单元格之前，请确保已运行 **安装包** 和 **导入与设置**。

**记得在进行下一步前停止此单元格。**


In [ ]:
# TODO: 在此处输入用于摘要的提示词
prompt_for_summarization = "请将以下文章概括成几句话。"

# 重置对话的函数
def reset() -> List:
    return []

# 调用模型生成摘要的函数
def interact_summarization(prompt: str, article: str, temp=1.0) -> List[Tuple[str, str]]:
    '''
    * 参数:
      - prompt: 我们在此部分中使用的提示词
      - article: 需要摘要的文章
      - temp: 模型的温度参数。温度用于控制聊天机器人的输出。温度越高，响应越具创造性。
    '''
    input = f"{prompt}\n{article}"
    response = client.chat.completions.create(
        model="qwen-turbo",  # 使用阿里云 DashScope 的模型
        messages=[{'role': 'user', 'content': input}],
        temperature=temp,
        max_tokens=200,  # 你需要注意到这里设置了文本的长度上限。
    )

    return [(input, response.choices[0].message.content)]

# 导出整个对话内容的函数
def export_summarization(chatbot: List[Tuple[str, str]], article: str) -> None:
    '''
    * 参数:
      - chatbot: 模型的对话记录，存储在元组列表中
      - article: 需要摘要的文章
    '''
    target = {"chatbot": chatbot, "article": article}
    with open("files/part1.json", "w") as file:
        json.dump(target, file)

# 生成 Gradio 的UI界面
with gr.Blocks() as demo:
    gr.Markdown("# 第1部分：摘要\n填写任何你喜欢的文章，让聊天机器人为你总结！")
    chatbot = gr.Chatbot()
    prompt_textbox = gr.Textbox(label="提示词", value=prompt_for_summarization, visible=False)
    article_textbox = gr.Textbox(label="文章", interactive=True, value="我家的后面有一个很大的园，相传叫作百草园。现在是早已并屋子一起卖给朱 文公的子孙了，连那最末次的相见也已经隔了七八年，其中似乎确凿只有一些野草 ；但那时却是我的乐园。 　　不必说碧绿的菜畦，光滑的石井栏，高大的皂荚树，紫红的桑椹；也不必说鸣 蝉在树叶里长吟，肥胖的黄蜂伏在菜花上，轻捷的叫天子（云雀）忽然从草间直窜 向云霄里去了。单是周围的短短的泥墙根一带，就有无限趣味。油蛉在这里低唱， 蟋蟀们在这里弹琴。翻开断砖来，有时会遇见蜈蚣；还有斑蝥，倘若用手指按住它 的脊梁，便会拍的一声，从后窍喷出一阵烟雾。何首乌藤和木莲藤缠络着，木莲有 莲房一般的果实，何首乌有拥肿的根。有人说，何首乌根是有象人形的，吃了便可 以成仙，我于是常常拔它起来，牵连不断地拔起来，也曾因此弄坏了泥墙，却从来 没有见过有一块根象人样。如果不怕刺，还可以摘到覆盆子，象小珊瑚珠攒成的小 球，又酸又甜，色味都比桑椹要好得远。 　 　　长的草里是不去的，因为相传这园里有一条很大的赤练蛇。 　　长妈妈曾经讲给我一个故事听：先前，有一个读书人住在古庙里用功，晚间， 在院子里纳凉的时候，突然听到有人在叫他。答应着，四面看时，却见一个美女的 脸露在墙头上，向他一笑，隐去了。他很高兴；但竟给那走来夜谈的老和尚识破了 机关。说他脸上有些妖气，一定遇见“美女蛇”了；这是人首蛇身的怪物，能唤人 名，倘一答应，夜间便要来吃这人的肉的。他自然吓得要死，而那老和尚却道无妨 ，给他一个小盒子，说只要放在枕边，便可高枕而卧。他虽然照样办，却总是睡不 着，——当然睡不着的。到半夜，果然来了，沙沙沙！门外象是风雨声。他正抖作 一团时，却听得豁的一声，一道金光从枕边飞出，外面便什么声音也没有了，那金 光也就飞回来，敛在盒子里。后来呢？后来，老和尚说，这是飞蜈蚣，它能吸蛇的 脑髓，美女蛇就被它治死了。 　　结末的教训是：所以倘有陌生的声音叫你的名字，你万不可答应他。　　 　　这故事很使我觉得做人之险，夏夜乘凉，往往有些担心，不敢去看墙上，而且 极想得到一盒老和尚那样的飞蜈蚣。走到百草园的草丛旁边时，也常常这样想。但 直到现在，总还没有得到，但也没有遇见过赤练蛇和美女蛇。叫我名字的陌生声音 自然是常有的，然而都不是美女蛇。 　　冬天的百草园比较的无味；雪一下，可就两样了。拍雪人（将自己的全形印在 雪上）和塑雪罗汉需要人们鉴赏，这是荒园，人迹罕至，所以不相宜，只好来捕鸟 。薄薄的雪，是不行的；总须积雪盖了地面一两天，鸟雀们久已无处觅食的时候才 好。扫开一块雪，露出地面，用一支短棒支起一面大的竹筛来，下面撒些秕谷，棒 上系一条长绳，人远远地牵着，看鸟雀下来啄食，走到竹筛底下的时候，将绳子一 拉，便罩住了。但所得的是麻雀居多，也有白颊的“张飞鸟”，性子很躁，养不过 夜的。 　　这是闰土的父亲所传授的方法，我却不大能用。明明见它们进去了，拉了绳， 跑去一看，却什么都没有，费了半天力，捉住的不过三四只。闰土的父亲是小半天 便能捕获几十只，装在叉袋里叫着撞着的。我曾经问他得失的缘由，他只静静地笑 道：你太性急，来不及等它走到中间去。 　　我不知道为什么家里的人要将我送进书塾里去了，而且还是全城中称为最严厉 的书塾。也许是因为拔何首乌毁了泥墙罢，也许是因为将砖头抛到间壁的梁家去了 罢，也许是因为站在石井栏上跳下来罢，……都无从知道。总而言之：我将不能常 到百草园了。Ａｄｅ，我的蟋蟀们！Ａｄｅ，我的覆盆子们和木莲们！ 　　出门向东，不上半里，走过一道石桥，便是我的先生的家了。从一扇黑油的竹 门进去，第三间是书房。中间挂着一块扁道：三味书屋；扁下面是一幅画，画着一 只很肥大的梅花鹿伏在古树下。没有孔子牌位，我们便对着那扁和鹿行礼。第一次 算是拜孔子，第二次算是拜先生。 　　第二次行礼时，先生便和蔼地在一旁答礼。他是一个高而瘦的老人，须发都花 白了，还戴着大眼镜。我对他很恭敬，因为我早听到，他是本城中极方正，质朴， 博学的人。 　　不知从那里听来的，东方朔也很渊博，他认识一种虫，名曰“怪哉”，冤气所 化，用酒一浇，就消释了。我很想详细地知道这故事，但阿长是不知道的，因为她 毕竟不渊博。现在得到机会了，可以问先生。 　　“先生，‘怪哉’这虫，是怎么一回事？……”我上了生书，将要退下来的时 候，赶忙问。 　　“不知道！”他似乎很不高兴，脸上还有怒色了。 　　我才知道做学生是不应该问这些事的，只要读书，因为他是渊博的宿儒，决不 至于不知道，所谓不知道者，乃是不愿意说。年纪比我大的人，往往如此，我遇见 过好几回了。 　　我就只读书，正午习字，晚上对课。先生最初这几天对我很严厉，后来却好起 来了，不过给我读的书渐渐加多，对课也渐渐地加上字去，从三言到五言，终于到 七言。 　　三味书屋后面也有一个园，虽然小，但在那里也可以爬上花坛去折腊梅花，在 地上或桂花树上寻蝉蜕。最好的工作是捉了苍蝇喂蚂蚁，静悄悄地没有声音。然而 同窗们到园里的太多，太久，可就不行了，先生在书房里便大叫起来：—— 　　“人都到那里去了？” 　　人们便一个一个陆续走回去；一同回去，也不行的。他有一条戒尺，但是不常 用，也有罚跪的规矩，但也不常用，普通总不过瞪几眼，大声道：—— 　　“读书！” 　　于是大家放开喉咙读一阵书，真是人声鼎沸。有念“仁远乎哉我欲仁斯仁至矣 ”的，有念“笑人齿缺曰狗窦大开”的，有念“上九潜龙勿用”的，有念“厥土下 上上错厥贡苞茅橘柚”的……先生自己也念书。后来，我们的声音便低下去，静下 去了，只有他还大声朗读着：—— 　　“铁如意，指挥倜傥，一座皆惊呢～～；金叵罗，颠倒淋漓噫，千杯未醉嗬～ ～……” 　　我疑心这是极好的文章，因为读到这里，他总是微笑起来，而且将头仰起，摇 着，向后面拗过去，拗过去。 　　先生读书入神的时候，于我们是很相宜的。有几个便用纸糊的盔甲套在指甲上 做戏。我是画画儿，用一种叫作“荆川纸”的，蒙在小说的绣像上一个个描下来， 象习字时候的影写一样。读的书多起来，画的画也多起来；书没有读成，画的成绩 却不少了，最成片断的是《荡寇志》和《西游记》的绣像，都有一大本。后来，因 为要钱用，卖给一个有钱的同窗了。他的父亲是开锡箔店的；听说现在自己已经做 了店主，而且快要升到绅士的地位了。这东西早已没有了罢。 　　　　　　　　　　　　　　　　　　 　　九月十八日。")
    
    with gr.Column():
        gr.Markdown("# 温度调节\n温度用于控制聊天机器人的输出。温度越高，响应越具创造性。")
        temperature_slider = gr.Slider(0.0, 2.0, 1.0, step=0.1, label="温度")
    
    with gr.Row():
        sent_button = gr.Button(value="发送")
        reset_button = gr.Button(value="重置")

    with gr.Column():
        gr.Markdown("# 保存结果\n当你对结果满意后，点击导出按钮保存结果。")
        export_button = gr.Button(value="导出")
    
    # 连接按钮与函数
    sent_button.click(interact_summarization, inputs=[prompt_textbox, article_textbox, temperature_slider], outputs=[chatbot])
    reset_button.click(reset, outputs=[chatbot])
    export_button.click(export_summarization, inputs=[chatbot, article_textbox])

# 启动 Gradio 界面
demo.launch(debug=True)

### 检查并打印你的结果

此部分用于检查你的“part1.json”文件是否包含所有正确的上下文，或者用于查看我们提供的示例文件。

你需要：

1. 确保文件列表中有你保存的 **part1.json** 文件。
2. 点击运行按钮。它将显示一个**冻结的** Gradio 界面，重现你的摘要结果。

---

在运行此单元格之前，请确保已运行 **安装库** 和 **导入与设置**。

**记得在进行下一步前停止此单元格。**

In [ ]:
# 加载对话记录的 JSON 文件
with open("files/part1.json", "r") as f:
    context = json.load(f)

chatbot = context['chatbot']  # 获取对话记录
article = context['article']  # 获取原始文章
summarization = chatbot[0][-1]  # 获取摘要结果

# 生成 Gradio 的UI界面
with gr.Blocks() as demo:
    gr.Markdown("# 第1部分：摘要\n你可以查看文章和摘要！")
    chatbot = gr.Chatbot(value=context['chatbot'])  # 加载对话历史
    article_textbox = gr.Textbox(label="文章", interactive=False, value=context['article'])  # 显示原始文章

    # 构建展示摘要和原文的部分
    with gr.Column():
        gr.Markdown("# 只是一个检查")
        gr.Textbox(label="文章", value=article, show_copy_button=True)  # 显示并允许复制原文
        gr.Textbox(label="摘要", value=summarization, show_copy_button=True)  # 显示并允许复制摘要

# 启动 Gradio 界面
demo.launch(debug=True)

## 第2部分：角色扮演（多轮对话应用）

在此任务中，你需要将聊天机器人设定为**角色扮演模式**。你应该为它指定一个角色，然后通过提示让它进入该角色的状态。

你需要完成以下步骤：

1. 想出一个你希望聊天机器人扮演的**角色**，以及一个使聊天机器人进入该角色的提示词。在 **character_for_chatbot** 中填写角色，在 **prompt_for_roleplay** 中填写提示词。
2. **点击运行按钮，界面将弹出一个可交互的界面。**
3. **与聊天机器人进行** 2 轮 **互动**。在标为“输入”的框中输入你想说的话，然后点击“发送”按钮。（你可以使用“温度”滑块来控制输出的创造性。）
4. 如果你**想更改提示词或角色**，可以停止单元格，返回TODO重新设置，然后重新运行单元格。
5. 在你获得满意的结果后，点击“导出”按钮保存结果。文件列表中将出现一个名为 **part2.json** 的文件。

注意：

- **如果你再次点击“导出”按钮，之前的结果将被覆盖。**
- **即使使用相同的提示词，输出的结果可能仍然不同。**

------

在运行此单元格之前，请确保已运行 **安装包** 和 **导入与设置**。

**记得在进行下一步前停止此单元格。**



In [ ]:
# TODO: 填写以下两行：character_for_chatbot 和 prompt_for_roleplay
# 第一个是你希望聊天机器人扮演的角色（注意，真正起作用的实际是prompt）
# 第二个是使聊天机器人扮演某个角色的提示词
character_for_chatbot = "面试官"
prompt_for_roleplay = "我需要你面试我有关AI的知识，仅提出问题"

# 清除对话的函数
def reset() -> List:
    return []

# 调用模型生成对话的函数
def interact_roleplay(chatbot: List[Tuple[str, str]], user_input: str, temp=1.0) -> List[Tuple[str, str]]:
    '''
    * 参数:

      - user_input: 每轮对话中的用户输入

      - temp: 模型的温度参数。温度用于控制聊天机器人的输出。温度越高，响应越具创造性。

    '''
    try:
        messages = []
        for input_text, response_text in chatbot:
            messages.append({'role': 'user', 'content': input_text})
            messages.append({'role': 'assistant', 'content': response_text})

        messages.append({'role': 'user', 'content': user_input})

        response = client.chat.completions.create(
            model="qwen-turbo",  # 使用阿里云 DashScope 的模型
            messages=messages,  # 包含用户的输入和对话历史
            temperature=temp,  # 使用温度参数控制创造性
            max_tokens=200,  # 控制输出的最大 token 数量
        )
        chatbot.append((user_input, response.choices[0].message.content))

    except Exception as e:
        print(f"发生错误：{e}")
        chatbot.append((user_input, f"抱歉，发生了错误：{e}"))
    return chatbot

# 导出整个对话记录的函数
def export_roleplay(chatbot: List[Tuple[str, str]], description: str) -> None:
    '''
    * 参数:

      - chatbot: 模型的对话记录，存储在元组列表中

      - description: 此任务的描述

    '''
    target = {"chatbot": chatbot, "description": description}
    with open("files/part2.json", "w") as file:
        json.dump(target, file)

# 进行第一次对话
first_dialogue = interact_roleplay([], prompt_for_roleplay)

# 生成 Gradio 的UI界面
with gr.Blocks() as demo:
    gr.Markdown(f"# 第2部分：角色扮演\n聊天机器人想和你玩一个角色扮演游戏，试着与它互动吧！")
    chatbot = gr.Chatbot(value=first_dialogue)
    description_textbox = gr.Textbox(label="机器人扮演的角色", interactive=False, value=f"{character_for_chatbot}")
    input_textbox = gr.Textbox(label="输入", value="")
    
    with gr.Column():
        gr.Markdown("# 温度调节\n温度用于控制聊天机器人的输出。温度越高，响应越具创造性。")
        temperature_slider = gr.Slider(0.0, 2.0, 1.0, step=0.1, label="温度")
    
    with gr.Row():
        sent_button = gr.Button(value="发送")
        reset_button = gr.Button(value="重置")
    
    with gr.Column():
        gr.Markdown("# 保存结果\n当你对结果满意后，点击导出按钮保存结果。")
        export_button = gr.Button(value="导出")

    # 连接按钮与函数
    sent_button.click(interact_roleplay, inputs=[chatbot, input_textbox, temperature_slider], outputs=[chatbot])
    reset_button.click(reset, outputs=[chatbot])
    export_button.click(export_roleplay, inputs=[chatbot, description_textbox])

# 启动 Gradio 界面
demo.launch(debug=True)

### 检查并打印你的结果

此部分用于检查你的 "part2.json" 文件是否包含所有正确的上下文信息，或者用于查看我们提供的示例文件。

你需要：

1. 确保文件列表中有你保存的 **part2.json** 文件。
2. 点击运行按钮。它将显示一个**冻结的** Gradio 界面，重现你的摘要结果。

------

在运行此单元格之前，请确保已运行 **安装库** 和 **导入与设置**。

**记得在进行下一步前停止此单元格。**


In [ ]:
# 加载对话记录的 JSON 文件
with open("files/part2.json", "r") as f:
    context = json.load(f)

# 遍历对话记录并正确存储
chatbot = context['chatbot']
role = context['description']  # 机器人扮演的角色
dialogue = ""  # 对话的完整记录
for i, (user, bot) in enumerate(chatbot):
    if i != 0:
        dialogue += f"用户: {user}\n"  # 用户输入
    dialogue += f"机器人: {bot}\n"  # 机器人回复

# 生成 Gradio 的 UI 界面
with gr.Blocks() as demo:
    gr.Markdown(f"# 第2部分：角色扮演\n聊天机器人和你玩了一个角色扮演游戏，查看结果吧！")
    chatbot = gr.Chatbot(value=context['chatbot'])  # 加载之前的对话记录
    description_textbox = gr.Textbox(label="机器人扮演的角色", interactive=False, value=context['description'])  # 显示角色
    with gr.Column():
        gr.Markdown("# 只是一个检查")
        gr.Textbox(label="角色", value=role, show_copy_button=True)  # 显示并允许复制角色信息
        gr.Textbox(label="对话", value=dialogue, show_copy_button=True)  # 显示并允许复制完整对话

# 启动 Gradio 界面
demo.launch(debug=True)

## 第3部分：定制化任务（多轮对话应用）

在本任务中，你需要提示聊天机器人执行某项特定任务。你应该首先设计一个你希望聊天机器人执行的任务，然后为它提供合适的提示词以引导其完成该任务。

你需要完成以下步骤：

1. 设计一个任务，并根据任务写一个提示词。将任务描述填写在 **chatbot_task** 中，并将提示词填写在 **prompt_for_task** 中。
2. **点击运行按钮**，与聊天机器人进行互动。在 "输入" 框中输入你的内容，然后点击 "发送" 按钮。（你可以使用 "温度" 滑块来控制输出的创造性。）
3. 如果你**想更改提示词或任务名称**，可以停止单元格，返回TODO重新设置，然后重新运行单元格。
4. 在你获得满意的结果后，点击 "导出" 按钮保存结果。文件列表中将出现一个名为 **part3.json** 的文件。

注意：

- **如果你再次点击 "导出" 按钮，之前的结果将被覆盖。**
- **即使使用相同的提示词，输出的结果可能仍然不同。**
- 实际上，这和第2部分是完全一致的，只是将提前预进行的prompt延后到了用户点击发送的时候，使得更像是一个通用的范例，隐藏了角色扮演的属性，但仅仅是隐藏，本质并没有改变。

---

在运行此单元格之前，请确保已运行 **安装包** 和 **导入与设置**。


In [ ]:
# TODO: 填写以下两行：chatbot_task 和 prompt_for_task，这里以翻译任务为例
# 第一个是用于告诉用户聊天机器人可以执行的任务（注意，真正起作用的实际是prompt）
# 第二个是使聊天机器人能够执行该任务的提示词
chatbot_task = "小学数学老师（输入“开始”）"
prompt_for_task = "现在开始，你将扮演一个出小学数学题的老师，当我说开始时提供一个简单的数学题，接收到正确回答后进行下一题，否则给我答案"

# 清除对话的函数
def reset() -> List:
    return []

# 调用模型生成对话的函数
def interact_customize(chatbot: List[Tuple[str, str]], prompt: str, user_input: str, temp=1.0) -> List[Tuple[str, str]]:
    '''
    * 参数:

      - chatbot: 模型本身，存储在元组列表中的对话记录

      - prompt: 用于指定任务的提示词

      - user_input: 每轮对话中的用户输入

      - temp: 模型的温度参数。温度用于控制聊天机器人的输出。温度越高，响应越具创造性。

    '''
    try:
        messages = []
        # 添加任务提示
        messages.append({'role': 'user', 'content': prompt})
        
        # 构建历史对话记录
        for input_text, response_text in chatbot:
            messages.append({'role': 'user', 'content': input_text})
            messages.append({'role': 'assistant', 'content': response_text})

        # 添加当前用户输入
        # 如果在这之前再次append prompt，就等价于在每次输入前都固定它的行为，这适用于比较呆的模型和prompt，比如【翻译成中文：】
        messages.append({'role': 'user', 'content': user_input})

        # 发送请求到对应的 API
        response = client.chat.completions.create(
            model="qwen-turbo",  # 使用阿里云 DashScope 的模型
            messages=messages,  # 传递消息记录
            temperature=temp,
            max_tokens=200,
        )

        # 将响应添加到对话记录中
        chatbot.append((user_input, response.choices[0].message.content))

    except Exception as e:
        print(f"发生错误：{e}")
        chatbot.append((user_input, f"抱歉，发生了错误：{e}"))
    
    return chatbot

# 导出对话记录的函数
def export_customized(chatbot: List[Tuple[str, str]], description: str) -> None:
    '''
    * 参数:

      - chatbot: 模型的对话记录，存储在元组列表中

      - description: 此任务的描述

    '''
    target = {"chatbot": chatbot, "description": description}
    with open("files/part3.json", "w") as file:
        json.dump(target, file)

# 生成 Gradio 的UI界面
with gr.Blocks() as demo:
    gr.Markdown("# 第3部分：定制化任务\n聊天机器人可以执行某项任务，试着与它互动吧！")
    chatbot = gr.Chatbot()
    desc_textbox = gr.Textbox(label="任务描述", value=chatbot_task, interactive=False)
    prompt_textbox = gr.Textbox(label="提示词", value=prompt_for_task, visible=False)
    input_textbox = gr.Textbox(label="输入", value="")
    
    with gr.Column():
        gr.Markdown("# 温度调节\n温度用于控制聊天机器人的输出。温度越高，响应越具创造性。")
        temperature_slider = gr.Slider(0.0, 2.0, 1.0, step=0.1, label="温度")
    
    with gr.Row():
        sent_button = gr.Button(value="发送")
        reset_button = gr.Button(value="重置")
    
    with gr.Column():
        gr.Markdown("# 保存结果\n当你对结果满意后，点击导出按钮保存结果。")
        export_button = gr.Button(value="导出")
    
    # 连接按钮与函数
    sent_button.click(interact_customize, inputs=[chatbot, prompt_textbox, input_textbox, temperature_slider], outputs=[chatbot])
    reset_button.click(reset, outputs=[chatbot])
    export_button.click(export_customized, inputs=[chatbot, desc_textbox])

# 启动 Gradio 界面
demo.launch(debug=True)

### 检查并打印你的结果

此部分用于检查你的 "part3.json" 文件是否包含所有正确的上下文信息，或者用于查看我们提供的示例文件。

你需要：

1. 确保文件列表中有你保存的 **part3.json** 文件。
2. 点击运行按钮。它将显示一个**冻结的** Gradio 界面，重现你的摘要结果。

------

在运行此单元格之前，请确保已运行 **安装库** 和 **导入与设置**。

**记得在进行下一步前停止此单元格。**


In [ ]:
# 加载对话记录的 JSON 文件
with open("files/part3.json", "r") as f:
    context = json.load(f)

# 遍历并加载对话记录
chatbot = context['chatbot']
desc = context['description']
dialogue = ""
for user, bot in chatbot:
    dialogue += f"用户: {user}\n"
    dialogue += f"机器人: {bot}\n"

# 生成 Gradio 的 UI 界面
with gr.Blocks() as demo:
    gr.Markdown("# 第3部分：定制化任务\n聊天机器人可以执行特定任务，试着与它互动吧！")
    chatbot = gr.Chatbot(value=context['chatbot'])  # 显示聊天记录
    desc_textbox = gr.Textbox(label="任务描述", value=context['description'], interactive=False)  # 显示任务描述
    
    with gr.Column():
        gr.Markdown("# 只是一个检查")
        gr.Textbox(label="任务描述", value=desc, show_copy_button=True)  # 显示并允许复制任务描述
        gr.Textbox(label="对话记录", value=dialogue, show_copy_button=True)  # 显示并允许复制对话记录

# 启动 Gradio 界面
demo.launch(debug=True)

## 拓展：阿里云的API格式，仅演示（不使用OpenAI的情况）

这里演示一下POST请求。

In [ ]:
# 导入所需的包
import requests
import gradio as gr
import json
from typing import List, 
    
    # 发送POST请求到DashScope API
    response = requests.post(DASHSCOPE_API_URL, headers=headers, json=data)
    
    # 如果请求成功，返回生成的内容
    if response.status_code == 200:
        result = response.json()
        return result.get('choices', [{}])[0].get('message', {}).get('content', '')
    else:
        # 捕捉并返回详细的错误信息
        error_message = f"错误: {response.status_code}, {response.text}"
        raise Exception(error_message)

# 检查是否正确设置了 DashScope API
# 如果一切正常，你将看到 "DashScope API 设置成功！！"
try:
    result = call_dashscope_api("测试")
    print("DashScope API 设置成功！！")
except Exception as e:
    print(f"DashScope API 可能有问题，请检查： {e}")